In [ ]:
# def segment_movements(df, max_dropped_frames=3):
#     segments = []
#     current_segment = []
#     nan_counter = 0
    
#     for i, row in df.iterrows():
#         if pd.isna(row['x']) or pd.isna(row['y']):  # Object not detected
#             nan_counter += 1
#         else:  # Object detected
#             nan_counter = 0
            
#         if nan_counter <= max_dropped_frames:  # Allowable dropped frames
#             current_segment.append(row)
#         else:
#             if current_segment:  # Save the current segment
#                 segments.append(pd.DataFrame(current_segment))
#                 current_segment = []  # Start a new segment
                
#     # Add last segment if any
#     if current_segment:
#         segments.append(pd.DataFrame(current_segment))
    
#     return segments

# movements = segment_movements(tongue_filtered, max_dropped_frames=3)

In [ ]:

def segment_movements(df, max_dropped_frames=3):
    segments = []
    current_segment = []
    nan_counter = 0
    
    for i, row in df.iterrows():
        if pd.isna(row['x']) or pd.isna(row['y']):  # Object not detected
            nan_counter += 1
        else:  # Object detected
            nan_counter = 0
            
        if nan_counter <= max_dropped_frames:  # Allowable dropped frames
            current_segment.append(row)
        else:
            if current_segment:  # Save the current segment if not empty
                segment_df = pd.DataFrame(current_segment)
                
                # Check if all values (except time) are NaN
                if not segment_df[['x', 'y', 'xv', 'yv', 'v']].isna().all().all():
                    # Trim only leading and trailing NaNs
                    first_valid_idx = segment_df[['x', 'y']].notna().idxmax().min()
                    last_valid_idx = segment_df[['x', 'y']].notna()[::-1].idxmax().min()
                    trimmed_segment_df = segment_df.loc[first_valid_idx:last_valid_idx].reset_index(drop=True)
                    
                    segments.append(trimmed_segment_df)
                
                current_segment = []  # Start a new segment
    
    # Add last segment if any and trim it
    if current_segment:
        segment_df = pd.DataFrame(current_segment)
        
        if not segment_df[['x', 'y', 'xv', 'yv', 'v']].isna().all().all():
            first_valid_idx = segment_df[['x', 'y']].notna().idxmax().min()
            last_valid_idx = segment_df[['x', 'y']].notna()[::-1].idxmax().min()
            trimmed_segment_df = segment_df.loc[first_valid_idx:last_valid_idx].reset_index(drop=True)
            segments.append(trimmed_segment_df)
    
    return segments


movements = segment_movements(tongue_filtered, max_dropped_frames=3)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_segmented_movements_global_time_colored(segments):
    plt.figure(figsize=(10, 6))
    
    # Find global minimum and maximum time for consistent color mapping
    all_times = pd.concat([segment['time'] for segment in segments])
    global_min_time = all_times.min()
    global_max_time = all_times.max()

    for i, segment in enumerate(segments):
        if not segment['x'].empty:
            # Normalize time based on the global min and max
            norm_time = (segment['time'] - global_min_time) / (global_max_time - global_min_time)
            norm_time = segment['time'] - segment['time'][0]
            
            # Mark start point of each segment
            plt.scatter(segment['x'].iloc[0], segment['y'].iloc[0], c = norm_time[0], cmap='YlGnBu_r', s=20, alpha=0.8)

            # Plot each segment with color gradient based on actual time
            plt.scatter(segment['x'], segment['y'], c=norm_time, cmap='YlGnBu_r', s=5,alpha = 0.8)

            
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.colorbar(label='Time (s)')
    plt.title('Segmented Tongue Movements')
    plt.show()

# Example usage
# Call plot_segmented_movements_global_time_colored on the list of segments from segment_movements function
plot_segmented_movements_global_time_colored(movements[0:50])


In [ ]:

# Function to plot original and segmented data with a categorical colormap
def plot_original_and_segmented_data(original_data, segments, xlim=None):
    plt.figure(figsize=(20, 6))

    # Plot original data
    plt.scatter(original_data['time'], original_data['x'], color='gray', s=30, label='Original Data', alpha=0.5)

    # Use 'tab20' colormap for distinct segment colors
    colors = plt.cm.tab20(np.arange(len(segments)) % 20)  # Cycle colors if more than 20 segments

    for i, segment in enumerate(segments):
        plt.scatter(segment['time'], segment['x'], color=colors[i], s=10)
    
    # Labels and legend
    plt.xlabel('Time')
    plt.ylabel('x position (pix)')
    plt.title('Original and Segmented Movements')
    
    if xlim is not None:
        plt.xlim(xlim)
    else:
        plt.xlim([0, segments[-1]['time'].values[-1]])

    plt.ylim([300,400])
    
    plt.show()


# Example usage
# Call plot_original_and_segmented_data with your original data and segments
plot_original_and_segmented_data(tongue_masked, movements[0:70],[0,60])
plot_original_and_segmented_data(tongue_masked, movements[0:70],[49,52])
plot_original_and_segmented_data(tongue_masked, movements[0:70],[50,50.25])


In [ ]:
#old kinematics filter that filtered all the position and velocity together. more principles to filter position sufficiently then derivative after filtering

def kinematics_filter(df, frame_rate=500, cutoff_freq=20, filter_order=8):
    """
    Applies interpolation and low-pass filtering to kinematic data to smooth trajectories and velocities.
    
    Parameters:
    df : pandas.DataFrame
        Input DataFrame containing time-series kinematic data with required columns: ['time', 'x', 'y'].
    frame_rate : int, optional
        Sampling frequency of the data in Hz (default is 500 Hz).
    cutoff_freq : float, optional
        Cutoff frequency for the low-pass Butterworth filter in Hz (default is 20 Hz).
    filter_order : int, optional
        Order of the Butterworth filter (default is 8).

    Returns:
    pandas.DataFrame
        A DataFrame with interpolated and filtered kinematic data, including time, position (x, y),
        velocity components (xv, yv), and speed (v).
    
    Notes:
    - Interpolates missing data points to ensure evenly spaced timestamps.
    - Computes velocity from positional changes over time.
    - Applies a zero-phase low-pass Butterworth filter to smooth kinematic signals.
    - Retains only the originally available (non-NaN) time points in the final output.
    """
    # Ensure the DataFrame has the required columns
    required_columns = ['time', 'x', 'y']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Input DataFrame must contain the columns: {required_columns}")
    
    # Generate new timestamps for interpolation
    t = df['time'].values
    dt = np.diff(t)
    new_ts = []
    for num in range(len(dt)):
        tspace = dt[num] / (1 / frame_rate)
        intgr = int(np.floor(tspace))
        if intgr >= 2:
            new_t = np.linspace(t[num], t[num + 1], intgr)
            new_ts.extend(new_t)
    new_t = np.unique(np.concatenate((t, new_ts)))
    
    # Interpolate missing data points
    x_nonan = df['x'][df['x'].notna()].values
    y_nonan = df['y'][df['y'].notna()].values
    t_nonan = df['time'][df['x'].notna()].values
    
    x = np.interp(new_t, t_nonan, x_nonan)
    y = np.interp(new_t, t_nonan, y_nonan)
    intrp = pd.DataFrame({'time': new_t, 'x': x, 'y': y})
    
    # Compute velocity
    times = intrp['time'].values
    t_diff = np.gradient(times)
    xv = np.gradient(intrp['x'].values) / t_diff
    yv = np.gradient(intrp['y'].values) / t_diff
    v = np.sqrt(xv**2 + yv**2)
    
    intrp['v'] = v
    intrp['xv'] = xv
    intrp['yv'] = yv
    
    # Apply low-pass Butterworth filter
    cutoff = cutoff_freq / (frame_rate / 2)
    b, a = butter(int(filter_order / 2), cutoff)  # Divide filter order by 2 for filtfilt
    filtered_values = filtfilt(b, a, intrp[['x', 'y', 'v', 'xv', 'yv']].values, axis=0)
    
    filtered_df = pd.DataFrame(filtered_values, columns=['x', 'y', 'v', 'xv', 'yv'])
    filtered_df.insert(0, 'time', intrp['time'].values)
    
    # Keep data only at original (non-NaN) timestamps
    df_temp = df.reindex(columns=list(filtered_df.columns.tolist()))
    df_nonan_index = df['time'][df['x'].notna()].index.tolist()
    filtered_df_nonan_index = filtered_df[filtered_df['time'].isin(t_nonan)].index.tolist()
    df_temp.iloc[df_nonan_index] = filtered_df.iloc[filtered_df_nonan_index]
    
    return df_temp.reset_index(drop=True)